In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NLP').master('local').getOrCreate()

In [2]:
data = spark.read.csv('SMSSpamCollection', inferSchema=True, sep='\t')

In [5]:
data.show(3)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
+----+--------------------+
only showing top 3 rows



In [6]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show(3)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
+-----+--------------------+
only showing top 3 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length', length(data['text']))
data.show(3)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
+-----+--------------------+------+
only showing top 3 rows



In [9]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [10]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [11]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

In [12]:
from pyspark.ml.feature import VectorAssembler

clean_up = VectorAssembler(
    inputCols=['tf_idf','length'],
    outputCol='features'
)

In [14]:
from pyspark.ml import Pipeline

data_prep_pipe = Pipeline(stages=[ham_spam_to_num, tokenizer, stopremove, count_vec, idf, clean_up])

In [15]:
cleaner = data_prep_pipe.fit(data)

In [16]:
clean_data = cleaner.transform(data)

In [17]:
clean_data.select('label','features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [18]:
clean_data = clean_data.select('label','features')

In [19]:
(training, testing) = clean_data.randomSplit([0.7, 0.3])

In [20]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

spam_predictor = nb.fit(training)

In [21]:
test_result = spam_predictor.transform(testing)

DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [25]:
test_result.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,4,50,...|[-826.35568464061...|[1.0,3.4087292923...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1005.0601357858...|[1.0,9.6144016450...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-820.78494114425...|[1.0,1.9982777084...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-879.89906285422...|[1.0,7.3874293448...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-556.17079320622...|[1.0,7.7617274879...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-556.17079320622...|[1.0,7.7617274879...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-873.20165946014...|[1.0,1.6103627017...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-692.80315452981...|[1.0,3.1102820856...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1287.8341138047...|[1.0,4.9818795756...|       0.0|
|  0.0|(13424,[0

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_result)
print('Accuracy of model at predicting spam was : {:.4f}'.format(acc))

Accuracy of model at predicting spam was : 0.9259
